In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Introduction to the course and loading our libraries

Hi and welcome to this notebook Which is made to help yall in doing GANs 

**what Does gans even stand for?**
Gan stands for Generative adversarial networks and pretty much means AI which makes Art
for more info go [here](https://en.wikipedia.org/wiki/Generative_adversarial_network) this is a   wikipedia articale explaining everything

**What language and what frameworks will we use?**
we will use Python with tensorflow since this is a simple intro course I will use the 'simplier' high level api Keras so that any one can get it easily if you want to learn via Pytorch this is another course which explains it pretty will [here is the course](https://www.youtube.com/watch?v=j3htLzG-Y_I)

**What will I learn on this 'intro course' ?**
this course will explain simple GANs we will build A custom model from scratch I will help you understand how everything works and I will have some refarals to understand the theory even more

**What data will we use?**
we will be using the data in [this](https://www.kaggle.com/competitions/gan-getting-started) competion (a competion made by the kaggle staff for GANs)

**Finally what should you do if you dont understand anything?**
If you dont get anything send me a message or even better just comment what you dont understand and tag me or you can just google it :)

Alr here we go GL and have fun :)

Lets now load our library

In [ ]:
import tensorflow as tf #this is loading tensorflow
from tensorflow.keras import utils,Sequential,layers
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Reshape, LeakyReLU, Dropout, UpSampling2D # Bring in the layers for the neural network
import matplotlib.pyplot as plt
#thats it will matplolib is option I will just use it for visualizition no more so if you arent proficeint in it dont even bother

# Loading and Preprocessing our data



lets now load the data there is a fancy way to do it (but I am too lazy) so I am just gonna use the built in [tf.keras.utils.image_dataset_from_directory()](https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory) cuz its easier

In [ ]:
true = utils.image_dataset_from_directory(
    '/kaggle/input/gan-getting-started',#this is the dir which has the data
    image_size=(256,256)#the size we want the image to be
)
#yes thats it really no joke we just loaded all the data (I will change one thing when training later on)

now we will normaliza the data by going through it

In [ ]:
def process(image,label):
    image = tf.cast(image/255. ,tf.float32) #this is the line that normilizes the data
    return image,label

true = true.map(process) #this goes through every single image and normilizes it

# Building the generating model

so the next part is kinda of hard i would recommend [this vid](https://www.youtube.com/watch?v=5g1eXmQtl0E) explains theortical part

note: I gain nothing from ANY of the sources i provide

In [ ]:
def build_generator(): 
    model = Sequential()
    
    # Takes in random values and reshapes it to 16,16,256
    # Beginnings of a generated image
    model.add(Dense(16*16, input_shape=(16,16,256)))
    model.add(LeakyReLU(0.2))
    model.add(Reshape((16,16,256)))
    
    # Upsampling block 1 
    model.add(UpSampling2D())
    model.add(Conv2D(128, 5, padding='same'))
    model.add(LeakyReLU(0.2))
    
    # Upsampling block 2 
    model.add(UpSampling2D())
    model.add(Conv2D(128, 5, padding='same'))
    model.add(LeakyReLU(0.2))
    
    # Convolutional block 3
    model.add(UpSampling2D())
    model.add(Conv2D(128, 4, padding='same'))
    model.add(LeakyReLU(0.2))
    
    # Convolutional block 4
    model.add(Conv2D(128, 4, padding='same'))
    model.add(LeakyReLU(0.2))
    
    # Conv layer to get to one channel
    model.add(UpSampling2D())
    model.add(Conv2D(3, 4, padding='same', activation='sigmoid'))
    
    return model
generator = build_generator()

In [ ]:
generator.summary()

lets now some cool prediction made by our not trained ai

In [ ]:
img = generator.predict(np.random.randn(4,16,16,256))
# Setup the subplot formatting 
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
# Loop four times and get images 
for idx, img in enumerate(img): 
    # Plot the image using a specific subplot 
    ax[idx].imshow(np.squeeze(img))
    # Appending the image label as the plot title 
    ax[idx].title.set_text(idx)

wow what an amazing art (JK) 

our model never was trained so thats makes some sense 
now lets make the detector 

# Building our dector model

In [ ]:
def build_discriminator(): 
    model = Sequential()
    
    # First Conv Block
    model.add(Conv2D(32, 5, input_shape = (256,256,3)))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    model.add(layers.MaxPool2D())

    # Second Conv Block
    model.add(Conv2D(64, 5))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    model.add(layers.MaxPool2D())
    # Third Conv Block
    model.add(Conv2D(128, 5))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    model.add(layers.MaxPool2D())
    # Fourth Conv Block
    model.add(Conv2D(256, 5))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    model.add(layers.MaxPool2D())
    model.add(Dropout(0.4))
    model.add(layers.MaxPool2D())
    model.add(Conv2D(256, 5))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    model.add(layers.MaxPool2D())

    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    
    return model 

In [ ]:
detector = build_discriminator()

In [ ]:
detector.summary()

In [ ]:
utils.plot_model(detector)

In [ ]:
utils.plot_model(generator)

will now were done the next part can be either the easiest or the hardest

easiest if you can easily build custom training function hardest if you can not

# training the model

code explination (generated by gpt 4) :
The code is about creating a special kind of model called a GAN. A GAN stands for Generative Adversarial Network. It is a way of making new things that look like real things, such as images, sounds, or texts. For example, a GAN can make new images of cats that look like real cats, but are not actually real cats.

A GAN has two parts: a generator and a discriminator. The generator’s job is to make new things, and the discriminator’s job is to tell if the things are real or fake. The generator and the discriminator are like two players in a game. The generator tries to fool the discriminator by making fake things that look real, and the discriminator tries to catch the generator by telling if the things are real or fake. The generator and the discriminator learn from each other and get better at their jobs over time.

The code defines a class called GAN_MODEL that inherits from another class called ks.models.Model. A class is like a blueprint that tells how to make something. In this case, the class tells how to make a GAN model.

The first part of the code is the __init__ method. This method is like a constructor that tells how to create an instance of the class. An instance is like an object that is made from the blueprint. In this case, the method tells how to create a GAN model object.

The method takes four arguments: generator, discriminator, *args, and **kwargs. The first two arguments are the generator and the discriminator models that are used to make the GAN model. The last two arguments are special symbols that mean any other arguments that are not specified.

The method calls another method called super().__init__(*args, **kwargs). This method is like calling the constructor of the parent class, which is ks.models.Model. This means that the GAN model object will have all the features of the parent class, plus some extra features that are defined in the child class.

The method then creates two attributes for the GAN model object: self.generator and self.discriminator. These attributes are like variables that store some values inside the object. In this case, they store the generator and the discriminator models that are passed as arguments.

The second part of the code is the compile method. This method is like a preparer that tells how to get ready for training the model. Training is like teaching the model how to do its job.

The method takes six arguments: genrator_optimezer, detector_optimizer, gen_loss, detector_loss, *args, and **kwargs. The first four arguments are some tools that are used to train the model. The last two arguments are again special symbols that mean any other arguments that are not specified.

The method calls another method called super().compile(*args, **kwargs). This method is like calling the preparer of the parent class, which is ks.models.Model. This means that the GAN model object will get ready for training using all the tools of the parent class, plus some extra tools that are defined in the child class.

The method then creates four more attributes for the GAN model object: self.g_opt, self.d_opt, self.g_loss, and self.d_loss. These attributes store the tools that are passed as arguments.

The third part of the code is the train_step method. This method is like a trainer that tells how to train the model for one step. A step is like one round of playing the game between the generator and the discriminator.

The method takes one argument: batch. This argument is like a bunch of data that is used to train the model. In this case, it is a bunch of real images.

The method does several things:

It gets some data by assigning real_images to be equal to batch. This means that it uses the bunch of real images as data.
It makes some new things by calling self.generator(tf.random.normal((input_shape)), training=False) and assigning it to be equal to fake_images. This means that it uses the generator model to make some fake images from some random numbers.
It trains the discriminator by doing these things:
It passes both real and fake images to the discriminator model by calling self.discriminator(real_images, training=True) and self.discriminator(fake_images, training=True) and assigning them to be equal to yhat_real and yhat_fake. These mean that it gets some predictions from the discriminator model about whether the images are real or fake.
It combines both predictions into one by calling tf.concat([yhat_real, yhat_fake], axis=0) and assigning it to be equal to yhat_realfake. This means that it puts the predictions together in one list.
It creates some labels for real and fake images by calling tf.concat([tf.zeros_like(yhat_real), tf.ones_like(yhat_fake)], axis=0) and assigning it to be equal to y_realfake. This means that it makes a list of zeros and ones, where zeros mean real and ones mean fake.
It adds some noise to the labels by doing these things:
It makes some random numbers between 0 and 0.15 by calling 0.15*tf.random.uniform(tf.shape(yhat_real)) and 0.15*tf.random.uniform(tf.shape(yhat_fake)) and assigning them to be equal to noise_real and noise_fake. This means that it makes some small numbers that are different for each prediction.
It adds the noise to the labels by calling tf.concat([noise_real, noise_fake], axis=0) and adding it to y_realfake. This means that it makes the labels a little bit different from zeros and ones, so that they are not too easy or too hard for the discriminator model to learn.
It calculates the loss by calling self.d_loss(y_realfake, yhat_realfake) and assigning it to be equal to total_d_loss. This means that it measures how well the discriminator model did its job by comparing the labels and the predictions. The loss is like a score that tells how much the discriminator model needs to improve.
It applies backpropagation by doing these things:
It calculates the gradients by calling d_tape.gradient(total_d_loss, self.discriminator.trainable_variables) and assigning it to be equal to dgrad. This means that it finds out how much each part of the discriminator model contributed to the loss.
It updates the discriminator model by calling self.d_opt.apply_gradients(zip(dgrad, self.discriminator.trainable_variables)). This means that it changes the parts of the discriminator model a little bit, so that they can do better next time.
It trains the generator by doing these things:
It makes some new images by calling self.generator(tf.random.normal((input_shape)), training=True) and assigning it to be equal to gen_images. This means that it uses the generator model to make some more fake images from some random numbers.
It creates some predicted labels by calling self.discriminator(gen_images, training=False) and assigning it to be equal to predicted_labels. This means that it gets some predictions from the discriminator model about whether the new fake images are real or fake.
It calculates the loss by calling self.g_loss(tf.zeros_like(predicted_labels), predicted_labels) and assigning it to be equal to total_g_loss. This means that it measures how well the generator model did its job by comparing the predictions with zeros. The loss is like a score that tells how much the generator model needs to improve. The generator model wants to make fake images that look real, so it wants the predictions to be close to zeros.
It applies backpropagation by doing these things:
It calculates the gradients by calling g_tape.gradient(total_g_loss, self.generator.trainable_variables) and assigning it to be equal to ggrad. This means that it finds out how much each part of the generator model contributed to the loss.
It updates the generator model by calling self.g_opt.apply_gradients(zip(ggrad, self.generator.trainable_variables)). This means that it changes the parts of the generator model a little bit, so that they can do better next time.
The method then returns a dictionary with two values: "detector_loss" and "generator_loss". A dictionary is like a container that stores some values with some names. In this case, it stores the losses of both models with their names.

The last part of the code is outside of the class definition. It does these things:

It creates a GAN model object called fashgan by calling GAN_MODEL(generator, detector). This means that it uses the blueprint of the class to make an object with a generator model and a discriminator model as arguments.
It prepares the GAN model object for training by calling fashgan.compile(genrator_optimezer, detector_optimizer, gen_loss, detector_loss). This means that it uses the preparer method of the class with some tools as arguments.
It checks if there are any callbacks or validation data by using some if-else statements. A callback is like a helper function that does something during training, such as saving or logging. A validation data is like a test data that is used


In [ ]:
from tensorflow import keras as ks
def training_model_for_GANs(genrator_optimezer, detector_optimizer, gen_loss, detector_loss,generator, detector,training_data,epochs,input_shape,Callbacks=None,val_data=None):
  class GAN_MODEL(ks.models.Model): 
      def __init__(self, generator, discriminator, *args, **kwargs):
          # Pass through args and kwargs to base class 
          super().__init__(*args, **kwargs)
          
          # Create attributes for gen and disc
          self.generator = generator 
          self.discriminator =  detector
          
      def compile(self, genrator_optimezer, detector_optimizer, gen_loss, detector_loss, *args, **kwargs): 
          # Compile with base class
          super().compile(*args, **kwargs)
          
          # Create attributes for losses and optimizers
          self.g_opt = genrator_optimezer
          self.d_opt = detector_optimizer
          self.g_loss = gen_loss
          self.d_loss = detector_loss

      def train_step(self, batch):
          # Get the data 
          real_images = batch
          fake_images = self.generator(tf.random.normal((input_shape)), training=False)
          
          # Train the discriminator
          with tf.GradientTape() as d_tape: 
              # Pass the real and fake images to the discriminator model
              yhat_real = self.discriminator(real_images, training=True) 
              yhat_fake = self.discriminator(fake_images, training=True)
              yhat_realfake = tf.concat([yhat_real, yhat_fake], axis=0)
              
              # Create labels for real and fakes images
              y_realfake = tf.concat([tf.zeros_like(yhat_real), tf.ones_like(yhat_fake)], axis=0)
              
              # Add some noise to the TRUE outputs
              noise_real = 0.15*tf.random.uniform(tf.shape(yhat_real))
              noise_fake = -0.15*tf.random.uniform(tf.shape(yhat_fake))
              y_realfake += tf.concat([noise_real, noise_fake], axis=0)
              
              # Calculate loss - BINARYCROSS 
              total_d_loss = self.d_loss(y_realfake, yhat_realfake)
              
          # Apply backpropagation - nn learn 
          dgrad = d_tape.gradient(total_d_loss, self.discriminator.trainable_variables) 
          self.d_opt.apply_gradients(zip(dgrad, self.discriminator.trainable_variables))
          
          # Train the generator 
          with tf.GradientTape() as g_tape: 
              # Generate some new images
              gen_images = self.generator(tf.random.normal((input_shape)), training=True)
                                          
              # Create the predicted labels
              predicted_labels = self.discriminator(gen_images, training=False)
                                          
              # Calculate loss - trick to training to fake out the discriminator
              total_g_loss = self.g_loss(tf.zeros_like(predicted_labels), predicted_labels) 
              
          # Apply backprop
          ggrad = g_tape.gradient(total_g_loss, self.generator.trainable_variables)
          self.g_opt.apply_gradients(zip(ggrad, self.generator.trainable_variables))
          
          return {"detector_loss":total_d_loss, "generator_loss":total_g_loss}
  fashgan = GAN_MODEL(generator, detector)
  fashgan.compile(genrator_optimezer, detector_optimizer, gen_loss, detector_loss)
  ## read before runing IF there are labels to your code (EG you loaded it from git hub or kaggle)
  training_data = training_data.as_numpy_iterator().next()
  ## and when your fitting use this
  ## fashgan.fit(training_data[0].....)
  if Callbacks == None and val_data==  None:
    return fashgan.fit(training_data[0],epochs=epochs)
  elif Callbacks == None and val_data != None:
    return fashgan.fit(training_data[0],epochs=epochs,validation_data=val_data)
  elif Callbacks != None and val_data == None:
    return fashgan.fit(training_data[0],epochs=epochs,callbacks=[Callbacks])
  else:
    return fashgan.fit(training_data[0],epochs=epochs,callbacks=[Callbacks],validation_data=val_data)

In [ ]:
#now lets train our model!!!!
training_model_for_GANs(genrator_optimezer = tf.keras.optimizers.Adam()
                        , detector_optimizer  = tf.keras.optimizers.Adam()
                        , gen_loss  = tf.keras.losses.binary_crossentropy
                        , detector_loss = tf.keras.losses.categorical_crossentropy
                        ,generator = generator
                        , detector = detector
                        ,training_data = true
                        ,epochs = 3
                        ,input_shape = (1,16, 16, 256)
                        ,Callbacks=None
                        ,val_data=None)

Obv I did low epochs cuz Its just an explination so i did so but you can try for longer and you can add callbacks if you want or val_data its all up to you

# predicting with our model

obv we trained on very low epoch (would suggest 2000+) but I am just explaining so the images will look awful but if you trained for longer it will get better obv

In [ ]:
img = generator.predict(np.random.randn(4,16,16,256))
# Setup the subplot formatting 
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
# Loop four times and get images 
for idx, img in enumerate(img): 
    # Plot the image using a specific subplot 
    ax[idx].imshow(np.squeeze(img))
    # Appending the image label as the plot title 
    ax[idx].title.set_text(idx)

wow something changed which is good

it actually looks not half bad tbh it looks 90% bad lol 😂😂

alr thats it hope that yall enjoyed this cool intro course and stay happy cia :D

if you still dont know how to use yolo v8 I have an intro course [here](https://www.kaggle.com/code/philopateergeorgei/intro-to-yolo-v8-for-object-detection)

If you like any of them toss me an upvote